In [2]:
# -*- coding: utf-8 -*-

#from shapely.geometry import LineString
import cv2
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from cv2 import imshow as cv2_imshow
import imutils
import math
import time
start_time = time.time()
from PIL import Image


In [3]:
def wrap_document(img):
  # Предварительная обработка изображения
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  blur = cv2.GaussianBlur(gray, (5, 5), 0)
  edges = cv2.Canny(blur, 50, 150)

  # Нахождение контуров на изображении
  contours, _ = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
  contours = sorted(contours, key=cv2.contourArea, reverse=True)

  # Определение контура, который представляет собой лист формата A4
  for contour in contours:
      # Вычисление приближенного контура
      perimeter = cv2.arcLength(contour, True)
      approx = cv2.approxPolyDP(contour, 0.02 * perimeter, True)

      # Если контур имеет 4 вершины, предполагаем, что это лист формата A4
      if len(approx) == 4:
          target = approx
          break

  # Применение перспективного преобразования
  pts = target.reshape(4, 2)
  rect = np.zeros((4, 2), dtype="float32")

  # Определение порядка точек: верхний левый, верхний правый, нижний правый, нижний левый
  s = pts.sum(axis=1)
  rect[0] = pts[np.argmin(s)]
  rect[2] = pts[np.argmax(s)]

  diff = np.diff(pts, axis=1)
  rect[1] = pts[np.argmin(diff)]
  rect[3] = pts[np.argmax(diff)]

  # Определение ширины и высоты результирующего изображения
  (tl, tr, br, bl) = rect
  widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
  widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
  heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
  heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))

  maxWidth = max(int(widthA), int(widthB))
  maxHeight = max(int(heightA), int(heightB))

  # Выполнение перспективного преобразования
  dst = np.array([
      [0, 0],
      [maxWidth - 1, 0],
      [maxWidth - 1, maxHeight - 1],
      [0, maxHeight - 1]], dtype="float32")

  transformMatrix = cv2.getPerspectiveTransform(rect, dst)
  warped = cv2.warpPerspective(img, transformMatrix, (maxWidth, maxHeight))

  return warped

In [4]:
def load_and_resize_image(image_path, width):
    """Загрузка и изменение размера изображения"""
    img = cv2.imread(image_path)
    resized_image = imutils.resize(img, width=width)
    img_wrapped = wrap_document(resized_image)
    return img_wrapped

In [5]:
def preprocess_image(img):
    """Предварительная обработка изображения"""
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred_frame = cv2.GaussianBlur(gray, (11, 11), 1)
    canny_frame = cv2.Canny(blurred_frame, 150, 150)
    return cv2.threshold(canny_frame, 100, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

In [6]:
def get_contours(thresh):
  """Получение контуров из изображения"""
  cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  return cnts[0] if len(cnts) == 2 else cnts[1]

In [7]:
def fill_with_color(cnts, img):
  # Создаем словарь сопоставления для числа вершин и цвета
  color_mapping = {
      3: (0, 255, 0),      # Зеленый = треугольник
      4: (0, 0, 255),      # Красный = квадрат
      5: (255, 0, 0),      # Синий = пятиугольник
      6: (255, 255, 0),    # Голубой = шестиугольник
      8: (255, 128, 255),  # Розовый = восьмиугольник
  }

  for cnt in cnts:
      approx = cv2.approxPolyDP(cnt, 0.05 * cv2.arcLength(cnt, True), True)
      vertices_count = len(approx)

      # Если контур соответствует одному из ключей в словаре
      if vertices_count in color_mapping:
          cv2.drawContours(img, [cnt], 0, color_mapping[vertices_count], -1)
      # Для большего числа вершин считаем контур кругом
      elif vertices_count > 12:
          cv2.drawContours(img, [cnt], 0, (0, 255, 255), -1)  # Желтый = круг

  return img

In [8]:
def get_biggest_area_2(cnts, can_check_limit_area = False):
    # Инициализация максимальной площади и массива для хранения наибольшего контура
    maxArea = 0
    biggest = []

    # Перебор всех контуров
    for i in cnts :
        # Вычисление площади контура
        area = cv2.contourArea(i)

        # Если площадь больше 100 или разрешена проверка ограниченной области
        if area > 100 or can_check_limit_area:
            # Вычисление периметра контура
            peri = cv2.arcLength(i, True)

            # Приближение формы контура
            edges = cv2.approxPolyDP(i, 0.05*peri, True)

            # Если площадь больше максимальной и контур имеет 4 вершины (предположительно прямоугольник)
            if area > maxArea and len(edges) == 4 :
                # Обновление наибольшего контура и максимальной площади
                biggest = edges
                maxArea = area

    # Возвращение наибольшего контура
    return biggest

In [9]:
# Функция для выделения и обработки секций изображения
def process_sections(image, coordinates):
    sectionsMark = []
    sectionThreshs = []
    biggestCounters = []
    filledSections = []

    for cordination in coordinates:
        cv2.rectangle(image, cordination[0], cordination[1], (255, 0, 0), 0)
        top, bottom, left, right = cordination[0][1], cordination[1][1], cordination[0][0], cordination[1][0]
        sectionsMark.append(image[top:bottom, left:right].copy())

    for sectionMark in sectionsMark:
        sectionThreshs.append(cv2.threshold(sectionMark, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1])

    for sectionThresh in sectionThreshs:
        cnts = cv2.findContours(sectionThresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        biggest = get_biggest_area_2(cnts)
        if len(biggest) > 0:
            biggestArea = cv2.contourArea(biggest)
            biggestCounters.append([biggest, biggestArea, sectionThresh])

    for counter in biggestCounters:
        resultMark = cv2.cvtColor(counter[2], cv2.COLOR_GRAY2BGR)
        fill_with_color([counter[0]], resultMark)
        # cv2_imshow(resultMark)
    return biggestCounters

In [10]:
def get_extreme_points(c, extreme):
  if extreme == 'left':
    return tuple(c[c[:, :, 0].argmin()][0])
  if extreme == 'right':
    return tuple(c[c[:, :, 0].argmax()][0])
  if extreme == 'top':
    return tuple(c[c[:, :, 1].argmin()][0])
  if extreme == 'bottom':
    return tuple(c[c[:, :, 1].argmax()][0])

In [11]:
def calculate_point(cordinate, counter, position):
  y = get_extreme_points(counter, position)[1]
  x = get_extreme_points(counter, position)[0]
  y += cordinate[1]
  x += cordinate[0]
  return (x, y)

In [12]:
def omr_area_coordinates(img):
  cordinates_of_react = [
    [(0, 210), (100, 300)], # left-1
    [(620,210), (715,300)], # right-2
    [(620,10), (715,100)], # right-1
    [(620,750), (715,840)], # right-3
    [(0,750), (100,840)], # left-2
    [(620,850), (715,1011)] # right-4
  ]

  # Вызов функции для обработки секций изображения
  biggest_counters = process_sections(img, cordinates_of_react)

  y=get_extreme_points(biggest_counters[0][0], 'bottom')[1]
  x=get_extreme_points(biggest_counters[0][0], 'right')[0]
  y=y+cordinates_of_react[0][0][1]
  x=x+cordinates_of_react[0][0][0]
  left_top_point = (x, y) # left-1

  y=get_extreme_points(biggest_counters[1][0], 'bottom')[1]
  x=get_extreme_points(biggest_counters[1][0], 'left')[0]
  y=y+cordinates_of_react[1][0][1]
  x=x+cordinates_of_react[1][0][0]
  right_top_point = (x, y) # right-2

  y=get_extreme_points(biggest_counters[1][0], 'bottom')[1]
  x=get_extreme_points(biggest_counters[1][0], 'right')[0]
  y=y+cordinates_of_react[1][0][1]
  x=x+cordinates_of_react[1][0][0]
  right_top_point1 = (x, y) # right-2

  y=get_extreme_points(biggest_counters[3][0], 'top')[1]
  x=get_extreme_points(biggest_counters[3][0], 'left')[0]
  y=y+cordinates_of_react[3][0][1]
  x=x+cordinates_of_react[3][0][0]
  right_bottom_point = (x, y) # right-3

  y=get_extreme_points(biggest_counters[4][0], 'top')[1]
  x=get_extreme_points(biggest_counters[4][0], 'right')[0]
  y=y+cordinates_of_react[4][0][1]
  x=x+cordinates_of_react[4][0][0]
  left_bottom_point = (x, y) # left-2

  min_x=left_top_point[0]
  max_x=right_top_point[0]
  min_y=right_top_point[1]
  max_y=right_bottom_point[1]

  return min_x, max_x, min_y, max_y

In [13]:
def get_counter_extreme_points(cnt):
  left = get_extreme_points(cnt, 'left')
  right = get_extreme_points(cnt, 'right')
  top = get_extreme_points(cnt, 'top')
  bottom = get_extreme_points(cnt, 'bottom')
  return left, right, top, bottom

In [14]:
def calculate_enclosing_circle(cnt):
  # Вычисление окружности, описывающей контур
  (x,y),radius = cv2.minEnclosingCircle(cnt)
  center = (int(x),int(y))
  radius = int(radius)
  return center, radius

In [15]:
def get_analyze_circles_cnts(img_org, min_x, max_x, min_y, max_y):
  img = img_org.copy()

  # Преобразование изображения в оттенки серого
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  # Применение размытия Гаусса
  blurred = cv2.GaussianBlur(gray, (5, 5), 0)

  # Бинаризация изображения
  _, thresh = cv2.threshold(blurred, 100, 255, cv2.THRESH_BINARY)

  # Поиск контуров
  contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  circle_counters = []
  for cnt in contours:
    # Вычисление моментов для определения центра масс
    M = cv2.moments(cnt)

    # Проверка, чтобы избежать деления на ноль
    if M["m00"] != 0:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
    else:
        cX, cY = 0, 0

    left, right, top, bottom = get_counter_extreme_points(cnt)

    # Проверяем, находятся ли крайние точки внутри заданного диапазона
    if left[0] < min_x or right[0] > max_x or top[1] < min_y or bottom[1] > max_y:
      continue

    # Вычисление соотношения сторон ограничивающего прямоугольника
    x,y,w,h = cv2.boundingRect(cnt)
    aspect_ratio = float(w)/h

    center, radius = calculate_enclosing_circle(cnt)

    # Вычисление площади контура и окружности
    area_cnt = cv2.contourArea(cnt)
    area_circle = np.pi * (radius**2)
    if area_circle != 0 and area_cnt > 20:
      if 0.7 < area_cnt/area_circle < 1.9:
        circle_counters.append([left, right, top, bottom])
  return circle_counters

In [16]:

def analyze_circles_and_draw(img_org, counters):
  img = img_org.copy()
  center, radius = calculate_enclosing_circle(cnt)

  # for i in counters:
  #   cv2.circle(img, center, radius, (0,255,0), -1)
  return img


In [17]:
def draw_intersect_vertical_lines(img):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  result_omr_area_coordinates = omr_area_coordinates(gray)
  min_x = result_omr_area_coordinates[0]
  max_x = result_omr_area_coordinates[1]
  min_y = result_omr_area_coordinates[2]
  max_y = result_omr_area_coordinates[3]
  extereme_counters = get_analyze_circles_cnts(img, min_x, max_x, min_y, max_y)
  height = img.shape[0]
  temp_img = img.copy()
  for item in extereme_counters:
    left_coordinate = item[0]
    left_point_x = left_coordinate[0]
    right_coordinate = item[1]
    right_point_x = right_coordinate[0]
    start_point = (left_point_x, 0)
    end_point = (right_point_x, height)
    temp_img = cv2.rectangle(temp_img, start_point, end_point, (0,250,0), -1)
  cv2_imshow(temp_img)

In [18]:
def draw_intersect_horizontal_lines(img):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  min_x, max_x, min_y, max_y = omr_area_coordinates(gray)
  extereme_counters = get_analyze_circles_cnts(img, min_x, max_x, min_y, max_y)
  width = img.shape[1]
  temp_img = img.copy()
  for item in extereme_counters:
    top_coordinate = item[2]
    top_point_y = top_coordinate[1]
    bottom_coordinate = item[3]
    bottom_point_y = bottom_coordinate[1]
    start_point = (0, top_point_y)
    end_point = (width, bottom_point_y)
    temp_img = cv2.rectangle(temp_img, start_point, end_point, (0,250,0), -1)
  cv2_imshow(temp_img)

In [19]:
img = load_and_resize_image("5.jpg", 720)
original = img.copy()
thresh = preprocess_image(img)
cnts = get_contours(thresh)

In [20]:
# Преобразование изображения из цветного в оттенки серого
gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)

# Применение порогового фильтра к изображению в оттенках серого
# cv2.THRESH_BINARY_INV инвертирует результирующее бинарное изображение
# cv2.THRESH_OTSU автоматически вычисляет оптимальное значение порога
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

# Инициализация номера области интереса (Region of Interest, ROI)
ROI_number = 0

# Поиск контуров на бинаризованном изображении
# cv2.RETR_EXTERNAL используется для извлечения только внешних контуров
# cv2.CHAIN_APPROX_SIMPLE удаляет все избыточные точки и сжимает контур, тем самым экономя память
cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# В зависимости от версии OpenCV, findContours возвращает разное количество объектов.
# Это условие обеспечивает совместимость кода с разными версиями OpenCV.
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

In [21]:
all_center_cnts = []

In [22]:
start_time=time.time()
for i in range(1,15):
  img = load_and_resize_image(f"5.jpg", 720)

  original = img.copy()
  thresh = preprocess_image(img)
  cnts = get_contours(thresh)
  # draw_intersect_vertical_lines(original)
  gray = cv2.cvtColor(original.copy(), cv2.COLOR_BGR2GRAY)
  result_omr_area_coordinates = omr_area_coordinates(gray)
  min_x = result_omr_area_coordinates[0]
  max_x = result_omr_area_coordinates[1]
  min_y = result_omr_area_coordinates[2]
  max_y = result_omr_area_coordinates[3]
  finded_circles_counters = get_analyze_circles_cnts(original, min_x, max_x, min_y, max_y)
  cv2.circle(original, (min_x, min_y), 3, (0,255,0), -1)
  cv2.circle(original, (max_x, min_y), 3, (0,255,0), -1)
  cv2.line(original, (max_x - 117, min_y), (max_x - 117, max_y), (0,255,0), 1)
  cv2.line(original, (max_x - 117*2, min_y), (max_x - 117*2, max_y), (0,255,0), 1)
  cv2.line(original, (max_x - 117*3, min_y), (max_x - 117*3, max_y), (0,255,0), 1)
  cv2.line(original, (max_x - 117*4, min_y), (max_x - 117*4, max_y), (0,255,0), 1)
  for cnt in finded_circles_counters:
    points = cnt
    cnt = np.array([list(p) for p in points], dtype=np.int32).reshape((-1, 1, 2))
    center, radius = calculate_enclosing_circle(cnt)
    cv2.circle(original, center, radius, (0,255,0), -1)
    # all_center_cnts.append(center)
  # cv2_imshow(original)
  # cv2.rectangle(original, (70, 300), (160,490), (0,250,0), 2)
  # cv2.rectangle(original, (190, 300), (285,490), (0,250,0), 2)
  # cv2.rectangle(original, (310, 300), (405,490), (0,250,0), 2)
  # cv2.rectangle(original, (435, 300), (530,774), (0,250,0), 2)
  # cv2.rectangle(original, (560, 300), (660,774), (0,250,0), 2)
  total_time = time.time() - start_time
  print("Общее время выполнения: {:.2f} секунд".format(total_time))
  start_time = time.time()
  for center in all_center_cnts:
    cv2.circle(original, center, 4, (0,255,255), -1)
    cv2_imshow(original)


Общее время выполнения: 0.05 секунд
Общее время выполнения: 0.04 секунд
Общее время выполнения: 0.05 секунд
Общее время выполнения: 0.05 секунд
Общее время выполнения: 0.04 секунд
Общее время выполнения: 0.04 секунд
Общее время выполнения: 0.05 секунд
Общее время выполнения: 0.04 секунд
Общее время выполнения: 0.04 секунд
Общее время выполнения: 0.04 секунд
Общее время выполнения: 0.04 секунд
Общее время выполнения: 0.04 секунд
Общее время выполнения: 0.04 секунд
Общее время выполнения: 0.04 секунд


In [23]:
start_ends = []
for i in range(1,15):
  img = load_and_resize_image(f"5.jpg", 720)

  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  # blurred_frame = cv2.GaussianBlur(gray, (11, 11), 1)
  canny_frame = cv2.Canny(gray, 150, 150)
  # cv2_imshow(gray)
  # cv2_imshow(blurred_frame)
  # cv2_imshow(canny_frame)

  result_omr_area_coordinates = omr_area_coordinates(gray)
  min_x = result_omr_area_coordinates[0]
  max_x = result_omr_area_coordinates[1]
  min_y = result_omr_area_coordinates[2]
  max_y = result_omr_area_coordinates[3]

  # Бинаризация изображения
  _, thresh = cv2.threshold(canny_frame, 100, 255, cv2.THRESH_BINARY_INV)
  # Создайте ядро для дилатации
  kernel = np.ones((3,7), np.uint8)

  erosion = cv2.erode(gray, kernel, iterations=3)
  # cv2_imshow(erosion)
  _, thresh = cv2.threshold(erosion, 120, 255, cv2.THRESH_BINARY)
  # cv2_imshow(thresh)
  blurred_frame = cv2.GaussianBlur(thresh, (11, 11), 12)
  blurred_frame = cv2.GaussianBlur(blurred_frame, (11, 11), 20)
  kernel = np.ones((15,10), np.uint8)
  erosion = cv2.erode(blurred_frame, kernel, iterations=1)
  _, thresh = cv2.threshold(erosion, 120, 255, cv2.THRESH_BINARY_INV)
  # cv2_imshow(thresh)
  contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  new_contours = []
  for cnt in contours:
    left, right, top, bottom = get_counter_extreme_points(cnt)

    # Проверяем, находятся ли крайние точки внутри заданного диапазона
    if left[0] < min_x or right[0] > max_x or top[1] < min_y or bottom[1] > max_y:
      continue
    new_contours.append(cnt)
  contours = new_contours
  print(len(contours))

  areas = []
  new_contrours = []
  start_end = []
  for cnt in contours:
    area_cnt = cv2.contourArea(cnt)
    areas.append(area_cnt)

    if area_cnt < 5000:
      continue

    left, right, top, bottom = get_counter_extreme_points(cnt)

    if min_y + (max_y - min_y)/2 < top[1]:
      continue

    cv2.rectangle(img, (left[0], top[1]), (right[0], bottom[1]), (0, 255, 0), 2)
    start_end.append([left[0], right[0]])
    # cv2.drawContours(img, [cnt], -1, (0,255,0), 1)
  start_ends.append(start_end)
  print(sorted(areas))
  ppp = [[69, 170],
  [187, 287],
  [296, 409],
  [424, 534],
  [540, 656]]
  height, width = img.shape[:2]
  for left, right in ppp:
    cv2.rectangle(img, (left, 0), (right, height), (0, 255, 255), 2)
    #cv2_imshow(img)
total_time = time.time() - start_time
print("Общее время выполнения: {:.2f} секунд".format(total_time))


15
[154.0, 414.0, 446.0, 727.0, 921.0, 963.5, 1112.5, 1532.5, 1706.5, 1759.0, 8059.5, 9114.5, 9566.5, 26493.0, 26618.0]
15
[154.0, 414.0, 446.0, 727.0, 921.0, 963.5, 1112.5, 1532.5, 1706.5, 1759.0, 8059.5, 9114.5, 9566.5, 26493.0, 26618.0]
15
[154.0, 414.0, 446.0, 727.0, 921.0, 963.5, 1112.5, 1532.5, 1706.5, 1759.0, 8059.5, 9114.5, 9566.5, 26493.0, 26618.0]
15
[154.0, 414.0, 446.0, 727.0, 921.0, 963.5, 1112.5, 1532.5, 1706.5, 1759.0, 8059.5, 9114.5, 9566.5, 26493.0, 26618.0]
15
[154.0, 414.0, 446.0, 727.0, 921.0, 963.5, 1112.5, 1532.5, 1706.5, 1759.0, 8059.5, 9114.5, 9566.5, 26493.0, 26618.0]
15
[154.0, 414.0, 446.0, 727.0, 921.0, 963.5, 1112.5, 1532.5, 1706.5, 1759.0, 8059.5, 9114.5, 9566.5, 26493.0, 26618.0]
15
[154.0, 414.0, 446.0, 727.0, 921.0, 963.5, 1112.5, 1532.5, 1706.5, 1759.0, 8059.5, 9114.5, 9566.5, 26493.0, 26618.0]
15
[154.0, 414.0, 446.0, 727.0, 921.0, 963.5, 1112.5, 1532.5, 1706.5, 1759.0, 8059.5, 9114.5, 9566.5, 26493.0, 26618.0]
15
[154.0, 414.0, 446.0, 727.0, 921.0, 9

In [24]:
def is_square(cnt):
  # Аппроксимируем контур
  epsilon = 0.02 * cv2.arcLength(cnt, True)
  approx = cnt
  # Если аппроксимированный контур имеет 4 вершины, это может быть квадрат или прямоугольник
  if len(approx) == 4:
      # Вычисляем площадь контура
      area = cv2.contourArea(cnt)

      # Вычисляем площадь ограничивающего прямоугольника
      x, y, w, h = cv2.boundingRect(approx)
      rect_area = w * h

      # Соотношение площадей
      area_ratio = area / rect_area

      # Соотношение сторон
      aspect_ratio = float(w) / h

      # Проверяем близость соотношений к 1
      if 0.95 < area_ratio < 1.05 and 0.95 < aspect_ratio < 1.05:
          # Проверяем равенство длин сторон
          (x1, y1), (x2, y2), (x3, y3), (x4, y4) = approx.ravel()
          edges = [cv2.norm((x1 - x2, y1 - y2)), cv2.norm((x2 - x3, y2 - y3)),
                    cv2.norm((x3 - x4, y3 - y4)), cv2.norm((x4 - x1, y4 - y1))]

          # Проверяем, все ли стороны примерно равны
          if max(edges) / min(edges) < 1.7:
              return True
              # Здесь может быть ваш код для обработки "квадратного" контура
  return True

In [25]:
for i in range(1,15):
  img = load_and_resize_image(f"5.jpg", 720)

  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  # blurred_frame = cv2.GaussianBlur(gray, (11, 11), 1)
  canny_frame = cv2.Canny(gray, 150, 150)
  _, thresh = cv2.threshold(canny_frame, 100, 255, cv2.THRESH_BINARY_INV)
  kernel = np.ones((5,1), np.uint8)

  erosion = cv2.erode(gray, kernel, iterations=3)
  kernel = np.ones((0,0), np.uint8)
  erosion = cv2.erode(erosion, kernel, iterations=1)
  _, thresh = cv2.threshold(erosion, 120, 255, cv2.THRESH_BINARY_INV)
  #cv2_imshow(thresh)
  continue
  contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  list_cnt_area = []
  for contour in contours:
    # Приближаем контуры с помощью многоугольников
    epsilon = 0.05 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)

    # Если контур имеет 4 вершины, возможно, это прямоугольник
    if len(approx) == 4:
        # Проверим является ли фигура выпуклой
        if cv2.isContourConvex(approx):
            area_cnt = cv2.contourArea(approx)
            if area_cnt < 500 and area_cnt > 300:
                if is_square(approx):
                    list_cnt_area.append(area_cnt)
                    cv2.drawContours(img, [approx], 0, (0, 255, 0), 1)
  print(sorted(list_cnt_area, reverse=True))
  print(img)
print("Общее время выполнения: {:.2f} секунд".format(total_time))


Общее время выполнения: 0.59 секунд


In [26]:
# print(start_ends.copy())
cp_start_ends = start_ends.copy()
block_1 = []
block_2 = []
block_3 = []
block_4 = []
block_5 = []
print(sorted(start_end, key=lambda x: x[0]))
for start_end in cp_start_ends:
  sorted_start_end = sorted(start_end, key=lambda x: x[0])
  block_1.append(sorted_start_end[0])
  block_2.append(sorted_start_end[1])
  block_3.append(sorted_start_end[2])
  block_4.append(sorted_start_end[3])
  block_5.append(sorted_start_end[4])

print([min(block_1, key=lambda x: x[0])[0], max(block_1, key=lambda x: x[1])[1]])
print([min(block_2, key=lambda x: x[0])[0], max(block_2, key=lambda x: x[1])[1]])
print([min(block_3, key=lambda x: x[0])[0], max(block_3, key=lambda x: x[1])[1]])
print([min(block_4, key=lambda x: x[0])[0], max(block_4, key=lambda x: x[1])[1]])
print([min(block_5, key=lambda x: x[0])[0], max(block_5, key=lambda x: x[1])[1]])


[[72, 156], [189, 274], [309, 393], [425, 513], [544, 632]]
[72, 156]
[189, 274]
[309, 393]
[425, 513]
[544, 632]


In [27]:
def find_rect_cnts(thresh):
  contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  finded_rects = []
  for cnt in contours:
      # Аппроксимируем контур
      epsilon = 0.02 * cv2.arcLength(cnt, True)
      approx = cv2.approxPolyDP(cnt, epsilon, True)

      # Если аппроксимированный контур имеет 4 вершины, это может быть квадрат или прямоугольник
      if len(approx) == 4:
          finded_rects.append(cnt)
  return finded_rects

In [28]:
def get_left_max_vertical_cnt(img):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  # blurred_frame = cv2.GaussianBlur(gray, (11, 11), 1)
  canny_frame = cv2.Canny(gray, 150, 150)
  _, thresh = cv2.threshold(canny_frame, 100, 255, cv2.THRESH_BINARY_INV)
  kernel = np.ones((4,1), np.uint8)

  erosion = cv2.erode(gray, kernel, iterations=3)
  kernel = np.ones((0,0), np.uint8)
  erosion = cv2.erode(erosion, kernel, iterations=1)
  _, thresh = cv2.threshold(erosion, 120, 255, cv2.THRESH_BINARY_INV)
  contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

  all_rect_cnts = contours
  # Сортировать контуры по убыванию высоты (получить прямоугольники и вытащить высоту)
  sorted_cnts_by_height = sorted(all_rect_cnts, key=lambda cnt: cv2.boundingRect(cnt)[3], reverse=True)

  # Взять первые пять контуров после сортировки
  top_5_height_contours = sorted_cnts_by_height[:5]

  # Теперь найти контур с максимальной площадью среди этих пяти
  max_area = 0
  max_area_contour = None

  for cnt in top_5_height_contours:
      area = cv2.contourArea(cnt)
      if area > max_area:
          max_area = area
          max_area_contour = cnt
  return max_area_contour
def remove_without_section_marks(img):
    max_area_contour = get_left_max_vertical_cnt(img)

    left, right, top, bottom = get_counter_extreme_points(max_area_contour)
    min_y = top[1] - 10
    max_y = bottom[1] + 10
    min_x = right[0]
    # Получите размеры изображения
    height, width = img.shape[:2]

    # Создайте маску, полностью заполненную нулями (черный цвет)
    mask = np.ones_like(img) * 0

    # Заполните область между min_y и max_y белым цветом
    mask[min_y:max_y, min_x:width] = (255,255,255)

    # Примените маску к изображению, чтобы оставить только область внутри min_y и max_y
    result = cv2.bitwise_and(img, mask)
    cv2.imshow('mask', mask)
    return result
def analyze_variant_columns(img):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  # blurred_frame = cv2.GaussianBlur(gray, (11, 11), 1)
  canny_frame = cv2.Canny(gray, 150, 150)
  _, thresh = cv2.threshold(canny_frame, 100, 255, cv2.THRESH_BINARY_INV)
  kernel = np.ones((2,2), np.uint8)

  erosion = cv2.erode(gray, kernel, iterations=3)
  kernel = np.ones((0,4), np.uint8)
  erosion = cv2.erode(erosion, kernel, iterations=1)
  _, thresh = cv2.threshold(erosion, 120, 255, cv2.THRESH_BINARY)
  cv2_imshow('thresh: ', thresh)
  # finded_rects = find_rect_cnts(thresh)
  # for cnt in finded_rects:
  #   cv2.drawContours(img, [cnt], 0, (0, 255, 255), -1)
cv2_imshow('img analyze_variant_columns func: ', img)

for i in range(1,15):
    img = load_and_resize_image(f"5.jpg", 720)
    result_img = remove_without_section_marks(img)
    cv2_imshow('result: ' ,result_img)
    
analyze_variant_columns(result_img)